In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [3]:
#sqlite 設定
con = sqlite3.connect('../../data/race_data_set.db') # データベースに接続する
con.enable_load_extension(True)
con.load_extension("../../../../sqlite-amalgamation-3260000/libsqlitefunctions.so")
cur = con.cursor()

In [82]:
sql_result = "select * from race_result order by horse_id,race_id"
sql_info   = "select * from race_info"
sql_pay = "select * from race_pay"

df_r = pd.read_sql_query(sql=sql_result,con=con)
df_i = pd.read_sql_query(sql=sql_info,con=con)

In [84]:
df_i.groupby("distance").count()

,race_id,race_num,race_name,race_date,race_rank1,race_rank2,kind,condition,place,horse_count,lap_time,conner1,conner2,conner3,conner4
distance,,,,,,,,,,,,,,,
1200,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,0,0,2299,2299
1300,99,99,99,99,99,99,99,99,99,99,99,0,0,99,99
1400,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,0,0,1579,1579
1500,36,36,36,36,36,36,36,36,36,36,36,0,36,36,36
1600,1292,1292,1292,1292,1292,1292,1292,1292,1292,1292,1292,0,307,1292,1292
1700,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795
1800,2859,2859,2859,2859,2859,2859,2859,2859,2859,2859,2859,2308,2493,2859,2859
1900,172,172,172,172,172,172,172,172,172,172,172,172,172,172,172
2000,1234,1234,1234,1234,1234,1234,1234,1234,1234,1234,1234,1058,1177,1234,1234


In [21]:
#次走の結果がどうなっているか
sql = "select * from race_result rr left join race_info ri on rr.race_id = ri.race_id"
df_r = pd.read_sql_query(sql=sql,con=con)
df_r["race_date"] = pd.to_datetime(df_r["race_date"],format='%Y-%m-%d')

In [26]:
df = pd.read_csv("../../data/horse_data_set2.csv")
df["pred_race_date"] = pd.to_datetime(df["pred_race_date"],format='%Y-%m-%d')
df["past_race_date"] = pd.to_datetime(df["past_race_date"],format='%Y-%m-%d')

In [29]:
data=df[df["past_race_num"]==1]

In [81]:
data[data["pred_race_id"]==201605040912][["horse_name","pred_kind","past_kind","pred_tyaku","past_tyaku","pred_place","past_place","pred_distance","past_distance"]]

,horse_name,pred_kind,past_kind,pred_tyaku,past_tyaku,pred_place,past_place,pred_distance,past_distance
110660,ジオラマ,ダ,ダ,11,3,東京,小倉,1600,1700
110683,クインズサターン,ダ,ダ,1,2,東京,東京,1600,1600
110704,シェアード,ダ,ダ,16,1,東京,中山,1600,1200
110717,クラーロデルナ,ダ,ダ,13,8,東京,中山,1600,1800
110720,シトラスクーラー,ダ,ダ,7,5,東京,中京,1600,1400
110744,アメリカンストーム,ダ,ダ,4,3,東京,東京,1600,1600
110746,サクラルコール,ダ,ダ,15,8,東京,東京,1600,1600
110761,タガノヴェリテ,ダ,ダ,5,3,東京,小倉,1600,1700
110771,ピットボス,ダ,ダ,3,1,東京,京都,1600,1800
110781,ラテラス,ダ,ダ,12,2,東京,東京,1600,1600


In [33]:
data[data["horse_name"]=="レッドアステル"]

,pred_race_id,past_race_id,horse_id,horse_name,past_race_num,pred_tyaku,pred_wakuban,pred_umaban,pred_horse_years,pred_horse_sex,...,past_race_date,past_race_rank1,past_race_rank2,past_distance,past_kind,past_condition,past_place,past_horse_count,past_lap_time,past_dif_time
865962,201901020309,201905040310,2016104670,レッドアステル,1.0,1,8,10,3,牝,...,2019-10-15,3歳以上,1000万下,1800,芝,良,東京,8,12.8-11.2-11.8-12.3-12.1-12.1-11.4-11.3-11.6,0.0
906551,201905040310,202006030310,2016104670,レッドアステル,1.0,1,1,1,3,牝,...,2020-04-04,4歳以上,1600万下,2200,芝,良,中山,17,12.4-11.5-12.9-13.0-13.0-12.5-12.3-11.7-11.5-1...,0.4
919876,201906020611,201909020611,2016104670,レッドアステル,1.0,2,7,13,3,牝,...,2019-04-07,3歳,オープン,1600,芝,良,阪神,18,12.2-11.1-12.1-12.3-11.7-10.8-11.0-11.5,2.0
978982,201909020611,201901020309,2016104670,レッドアステル,1.0,14,8,17,3,牝,...,2019-08-24,3歳以上,500万下,1800,芝,稍重,札幌,10,12.9-12.2-12.6-12.5-12.5-12.4-12.0-11.6-11.8,0.0


In [65]:
for i,j in data.groupby("pred_race_id"):
    if np.mean(j.past_tyaku.values)<4.3:
        print(i,np.mean(j.past_tyaku.values))

201602010608 4.2727272727272725
201605021105 3.3333333333333335
201605040909 4.111111111111111
201605040912 3.933333333333333
201606020409 4.111111111111111
201607020609 3.3333333333333335
201608010710 4.25
201608020407 4.0
201608020707 3.4166666666666665
201608030104 4.235294117647059
201608030109 4.285714285714286
201608030212 4.2
201608040510 4.285714285714286
201609020309 3.727272727272727
201609020506 3.8461538461538463
201609040303 2.5
201609040408 3.2
201609040409 4.1
201609040611 3.2142857142857144
201701020111 4.181818181818182
201701020202 3.6
201702010312 4.0
201702020208 3.4444444444444446
201702020603 3.875
201704030111 3.7142857142857144
201705020505 3.625
201705020811 4.0625
201705030511 4.2
201706010202 3.909090909090909
201706040511 4.285714285714286
201707020307 4.230769230769231
201708010209 4.111111111111111
201708010610 3.0
201708020607 4.0
201708030109 3.6666666666666665
201708030305 4.222222222222222
201708031009 4.0
201709010308 4.090909090909091
201709010807 4.

In [74]:
data[data["past_tyaku"]<4].groupby("pred_race_id").count().sort_values(by="horse_id", ascending=False)

,past_race_id,horse_id,horse_name,past_race_num,pred_tyaku,pred_wakuban,pred_umaban,pred_horse_years,pred_horse_sex,pred_race_date,...,past_race_date,past_race_rank1,past_race_rank2,past_distance,past_kind,past_condition,past_place,past_horse_count,past_lap_time,past_dif_time
pred_race_id,,,,,,,,,,,,,,,,,,,,,
201609040408,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
201605040912,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
201609020710,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
201805020511,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
201808050812,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201809010605,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
201606030305,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
201809010702,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


4.0